In [ ]:
# %matplotlib widget
import urllib

import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
import plotly.graph_objects as go
import polars as pl
import pyproj
import requests
from brdr.aligner import Aligner
from brdr.enums import OpenbaarDomeinStrategy
from brdr.loader import DictLoader
from matplotlib.widgets import Slider
from shapely import MultiPolygon
from shapely.ops import unary_union
from shapely.wkt import loads

In [ ]:
datasette_base_url = "https://datasette.planning.data.gov.uk/conservation-area.csv"

query = """
select * 
from entity
"""
encoded_query = urllib.parse.urlencode({"sql": query})

r = requests.get(f"{datasette_base_url}?{encoded_query}", auth=("user", "pass"))

filename = "datasette_data.csv"
with open(filename, "wb") as f_out:
    f_out.write(r.content)

data = pl.read_csv(filename)

In [ ]:
def download_osm_polygons(
    polygon_for_osm,
    osm_tags,
    osm_query_crs,
    plot_crs,
    line_buffer,
):
    # Download Open Street Map polygons
    osm_features_df = gpd.GeoDataFrame(geometry=[], crs=osm_query_crs)
    osm_features_base = ox.features_from_polygon(polygon_for_osm, osm_tags)
    osm_features_df = osm_features_base[
        osm_features_base.geometry.geom_type.isin(
            ["Polygon", "MultiPolygon", "LineString"]
        )
    ]
    # This ensures that LineStrings are also interpreted as v.small polygons - shortcut for all in one solution.
    osm_features_df = osm_features_df.to_crs(plot_crs)
    line_mask = osm_features_df["geometry"].geom_type == "LineString"
    osm_features_df.loc[line_mask, "geometry"] = osm_features_df.loc[
        line_mask, "geometry"
    ].buffer(line_buffer)

    osm_features_df = osm_features_df.buffer(
        0
    )  # Buffer of zero is to ensure GeoPandas series, rather than df with everything else.
    print(f"Downloaded {len(osm_features_df)} OSM polygons.")

    # Align OSM co-ordinate systems
    if not osm_features_df.empty:
        if osm_features_df.crs == plot_crs:
            osm_features_proj = osm_features_df
        else:
            osm_features_proj = osm_features_df.to_crs(plot_crs)
    else:
        osm_features_proj = gpd.GeoDataFrame(geometry=[], crs=plot_crs)
    return osm_features_proj

In [ ]:
def process_osm_polygons(osm_features_proj, used_osm_indices):
    reference_geom = None

    if not osm_features_proj.empty:
        geometries_to_combine = None

        if used_osm_indices is None:
            # Case 1: Use ALL downloaded features
            print("Using all polygons.")
            geometries_to_combine = osm_features_proj.geometry
        elif isinstance(used_osm_indices, list) and used_osm_indices:
            # Case 2: Use features at specific indices
            print(f"Selecting polygons at indices {used_osm_indices}.")
            geometries_to_combine = osm_features_proj.iloc[used_osm_indices].geometry

        # Combine if geometries were selected
        if geometries_to_combine is not None and not geometries_to_combine.empty:
            # Combine all the geometries we need
            combined_reference = unary_union(geometries_to_combine)  # .buffer(0)
            if not combined_reference.is_empty:
                reference_geom = combined_reference
            else:
                print("Empty osm polygons when combining")
        else:
            print("Indices may be out of scope")

    else:
        print("No OSM polygons downloaded.")
    return reference_geom

In [ ]:
def get_snapped_polygon(
    reference_geom,
    original_proj,
    brdr_distance,
    brdr_strategy,
    brdr_threshold,
    plot_crs,
):
    aligned_df = None
    if reference_geom is not None and not original_proj.empty:
        thematic_geom = original_proj["geometry"].iloc[0]
        if thematic_geom.is_valid or thematic_geom.buffer(0).is_valid:
            if not thematic_geom.is_valid:
                thematic_geom = thematic_geom.buffer(0)

            # alignment logic
            aligner = Aligner(crs=plot_crs)
            loader_thematic = DictLoader({"theme_id_1": thematic_geom})
            aligner.load_thematic_data(loader_thematic)
            loader_reference = DictLoader({"ref_id_1": reference_geom})
            aligner.load_reference_data(loader_reference)

            process_result = aligner.process(
                relevant_distance=brdr_distance,
                od_strategy=brdr_strategy,
                threshold_overlap_percentage=brdr_threshold,
            )
            aligned_geom = process_result["theme_id_1"][brdr_distance]["result"]
            diff_geom = process_result["theme_id_1"][brdr_distance]["result_diff"]
            if not aligned_geom.is_valid:
                aligned_geom = aligned_geom.buffer(0)

            aligned_df = gpd.GeoDataFrame([1], geometry=[aligned_geom], crs=plot_crs)

            if not diff_geom.is_valid:
                diff_geom = diff_geom.buffer(0)

            diff_df = gpd.GeoDataFrame([1], geometry=[diff_geom], crs=plot_crs)

    return aligned_df, diff_df, process_result

In [ ]:
def plot_all_areas(
    aligned_df,
    diff_df,
    original_proj,
    osm_features_proj,
    osm_tags,
    plot_crs,
):
    fig, ax = plt.subplots(figsize=(12, 12))

    plot_layers_for_bounds = [original_proj]
    if aligned_df is not None:
        plot_layers_for_bounds.append(aligned_df)
    combined_bounds_df = pd.concat(
        [df for df in plot_layers_for_bounds if not df.empty], ignore_index=True
    )

    plot_xlim, plot_ylim = None, None
    if not combined_bounds_df.empty:
        xmin, ymin, xmax, ymax = combined_bounds_df.total_bounds
        if xmax > xmin and ymax > ymin:  # Basic check
            x_padding = (xmax - xmin) * 0.1
            y_padding = (ymax - ymin) * 0.1
            plot_xlim = (xmin - x_padding, xmax + x_padding)
            plot_ylim = (ymin - y_padding, ymax + y_padding)

    # Plot original area
    original_proj.boundary.plot(
        ax=ax,
        color="black",
        linestyle="--",
        linewidth=2.0,
        label="Original Area",
        zorder=2,
        alpha=0.8,
    )

    # Plot new boundary
    if aligned_df is not None:
        aligned_df.boundary.plot(
            ax=ax, color="red", linewidth=2.0, label="New Area", zorder=4, alpha=0.9
        )

    # Plot OSM polygons
    if not osm_features_proj.empty:
        osm_features_proj.plot(
            ax=ax,
            label=f"OSM Features ({list(osm_tags.keys())[0]})",
            zorder=3,
            edgecolor="blue",
            facecolor="lightblue",
            linewidth=0.5,
            alpha=0.6,
        )
        combined_osm = unary_union(osm_features_proj).intersection(diff_df["geometry"])
        changed_features_ratio = (
            100 * combined_osm.area / unary_union(osm_features_proj).area
        )
        changed_total_ratio = 100 * combined_osm.area / aligned_df.area
        print(f"percentage of feature area changed: {changed_features_ratio[0]}%")
        print(
            f"changed area as percentage of total boundary: {changed_total_ratio[0]}%"
        )
        combined_osm.plot(
            ax=ax,
            label="Difference in plots",
            zorder=3,
            edgecolor="darkmagenta",
            facecolor="magenta",
            linewidth=0.5,
            alpha=0.6,
        )
    # Set Limits (if calculated)
    if plot_xlim and plot_ylim:
        ax.set_xlim(plot_xlim)
        ax.set_ylim(plot_ylim)

    # Add Basemap
    ctx.add_basemap(
        ax,
        # source=ctx.providers.OpenStreetMap.Mapnik,
        source=ctx.providers.Esri.WorldImagery,
        crs=plot_crs,
        zoom="auto",
        zorder=1,
    )

    # Final Appearance
    ax.set_axis_off()
    ax.set_title(
        f"Original & Aligned Area with base Features ({osm_tags})\n(Plot CRS: {plot_crs})"
    )
    ax.legend(loc="best")  # Adjust legend location if needed
    plt.tight_layout()
    plt.show()
    print("Plot displayed.")

In [ ]:
def align_and_plot_area(
    original_wkt: str,
    initial_crs: str = "EPSG:4326",
    osm_tags: dict | None = None,
    brdr_distance: float = 20,
    brdr_threshold: float = 10,
    brdr_strategy=OpenbaarDomeinStrategy.SNAP_ONLY_VERTICES,
    plot_crs: str = "EPSG:3857",
    osm_query_crs: str = "EPSG:4326",
    used_osm_indices: list | None = None,
    polygon_detection_buffer: float = 0.00001,
    line_buffer: float = 0.00001,
):

    # Load datasette polygon for area
    original_geom = loads(original_wkt)
    original_df = gpd.GeoDataFrame([1], geometry=[original_geom], crs=initial_crs)
    adding_buffer_df = original_df.to_crs(plot_crs).boundary.buffer(
        polygon_detection_buffer
    )
    # polygon_for_osm = original_df
    # Ensures correct crs
    # df_for_query = original_df.to_crs(osm_query_crs)
    df_for_query = adding_buffer_df.to_crs(osm_query_crs)

    # Need to add a small buffer to ensure that all nearby features are captured. May need testing.
    # polygon_for_osm = df_for_query.geometry.iloc[0].boundary.buffer(polygon_detection_buffer) # Changed .boundary here to avoid getting inner ones
    polygon_for_osm = df_for_query.geometry.iloc[
        0
    ]  # Changed .boundary here to avoid getting inner ones

    # if original_df.crs == plot_crs:
    #     original_proj = original_df
    # else:
    original_proj = original_df.to_crs(plot_crs)

    # polygon_for_osm = polygon_for_osm.buffer(polygon_detection_buffer) # Changed .boundary here to avoid getting inner ones

    # # Download Open Street Map polygons
    osm_features_proj = download_osm_polygons(
        polygon_for_osm=polygon_for_osm,
        osm_tags=osm_tags,
        osm_query_crs=osm_query_crs,
        plot_crs=plot_crs,
        line_buffer=line_buffer,
    )

    # Prep natural features
    reference_geom = process_osm_polygons(osm_features_proj, used_osm_indices)

    # Brdr for snapping polygons
    aligned_df, diff_df, process_result = get_snapped_polygon(
        reference_geom=reference_geom,
        original_proj=original_proj,
        brdr_distance=brdr_distance,
        brdr_strategy=brdr_strategy,
        brdr_threshold=brdr_threshold,
        plot_crs=plot_crs,
    )

    original_border = original_df["geometry"].to_crs("EPSG:3857")[0]
    new_border = aligned_df["geometry"][0]
    base_features = MultiPolygon(list(osm_features_proj))
    difference_area = base_features.intersection(diff_df["geometry"])
    difference_area = difference_area.explode()
    difference_area = difference_area[
        difference_area.geometry.geom_type.isin(["Polygon", "MultiPolygon"])
    ]
    difference_area = MultiPolygon(list(difference_area))

    return original_border, new_border, difference_area, base_features

In [ ]:
# def plot_area_with_sliders(
#         original_border,
#         new_border,
#         difference_area,
#         diff_rgb,
#         base_features,
#         base_rgb,
#         alpha,
#         source_crs = "EPSG:3857",
#         target_crs = "EPSG:4326",
# ):

# plot_area_with_sliders(
#         original_border,
#         aligned_result_df['geometry'][0],
#         combined_osm,
#         (255,0,0),
#         osm_features_proj,
#         (0,0,255),
#         0.6,
#     )

In [ ]:
def plot_all_areas(
    aligned_df,
    diff_df,
    original_proj,
    osm_features_proj,
    osm_tags,
    plot_crs,
    plot_sliders,
):
    fig, ax = plt.subplots(figsize=(10, 10))
    feature_collection = None

    plot_layers_for_bounds = [original_proj]
    if aligned_df is not None:
        plot_layers_for_bounds.append(aligned_df)
    combined_bounds_df = pd.concat(
        [df for df in plot_layers_for_bounds if not df.empty], ignore_index=True
    )

    plot_xlim, plot_ylim = None, None
    if not combined_bounds_df.empty:
        xmin, ymin, xmax, ymax = combined_bounds_df.total_bounds
        if xmax > xmin and ymax > ymin:  # Basic check
            x_padding = (xmax - xmin) * 0.1
            y_padding = (ymax - ymin) * 0.1
            plot_xlim = (xmin - x_padding, xmax + x_padding)
            plot_ylim = (ymin - y_padding, ymax + y_padding)

    # Plot original area
    original_proj.boundary.plot(
        ax=ax,
        color="black",
        linestyle="--",
        linewidth=2.0,
        label="Original Area",
        # zorder=2,
        alpha=0.8,
    )

    # Plot new boundary
    if aligned_df is not None:
        aligned_df.boundary.plot(
            ax=ax, color="red", linewidth=2.0, label="New Area", zorder=4, alpha=0.9
        )

    feature_alpha = 0.5
    # Plot OSM polygons
    if not osm_features_proj.empty:
        osm_features_proj.plot(
            ax=ax,
            label=f"OSM Features ({list(osm_tags.keys())[0]})",
            edgecolor="blue",
            facecolor="lightblue",
            linewidth=0.5,
            alpha=feature_alpha,
        )

    if ax.collections:
        feature_collection = ax.collections[-1]

        combined_osm = unary_union(osm_features_proj).intersection(diff_df["geometry"])
        changed_features_ratio = (
            100 * combined_osm.area / unary_union(osm_features_proj).area
        )
        changed_total_ratio = 100 * combined_osm.area / aligned_df.area
        print(f"percentage of feature area changed: {changed_features_ratio[0]}%")
        print(
            f"changed area as percentage of total boundary: {changed_total_ratio[0]}%"
        )
        combined_osm.plot(
            ax=ax,
            label="Difference in plots",
            edgecolor="darkmagenta",
            facecolor="magenta",
            linewidth=0.5,
            alpha=0.6,
        )
    # Set Limits (if calculated)
    if plot_xlim and plot_ylim:
        ax.set_xlim(plot_xlim)
        ax.set_ylim(plot_ylim)

    # Add Basemap
    ctx.add_basemap(
        ax,
        # source=ctx.providers.OpenStreetMap.Mapnik,
        source=ctx.providers.Esri.WorldImagery,
        crs=plot_crs,
        zoom="auto",
        # zorder=1,
    )

    if plot_sliders:

        def update_osm_alpha(val):
            feature_collection.set_alpha(val)
            fig.canvas.draw_idle()

        ax_feature_slider = fig.add_axes(
            [0.25, 0.05, 0.5, 0.03], facecolor="lightgoldenrodyellow"
        )
        feature_slider = Slider(
            ax=ax_feature_slider,
            label="OSM Alpha",
            valmin=0.0,
            valmax=1.0,
            valinit=feature_alpha,  # Start slider at the initial alpha
        )

        feature_slider.on_changed(update_osm_alpha)
    # fig._feature_slider = feature_slider

    # Final Appearance
    ax.set_axis_off()
    ax.set_title(
        f"Original & Aligned Area with base Features ({osm_tags})\n(Plot CRS: {plot_crs})"
    )
    ax.legend(loc="best")  # Adjust legend location if needed
    # plt.tight_layout()
    plt.tight_layout(rect=[0, 0.08, 1, 1])
    plt.show()
    print("Plot displayed.")

In [ ]:
input_tags = {
    "landuse": ["residential", "farmyard"],
    # 'landuse': ['farmyard'],
    # 'landuse': ['farmland'],
    # 'highway': ['track'],
    # 'waterway': ['drain'],
    # "natural": ["wood"]
    # "highway": ['primary', 'secondary', 'unclassified']
}
input_brdr_distance = 30
input_brdr_threshold = 100
# input_brdr_strategy = OpenbaarDomeinStrategy.SNAP_ONLY_VERTICES
input_brdr_strategy = OpenbaarDomeinStrategy.SNAP_PREFER_VERTICES
initial_crs = "EPSG:4326"
target_crs = "EPSG:3857"  # CRS for brdr and plotting
target_osm_crs = "EPSG:4326"  # CRS for osm input polygon

original_border, new_border, difference_area, base_features = align_and_plot_area(
    original_wkt=data["geometry"][0],
    initial_crs=initial_crs,
    osm_tags=input_tags,
    brdr_distance=input_brdr_distance,
    brdr_threshold=input_brdr_threshold,
    brdr_strategy=input_brdr_strategy,
    plot_crs=target_crs,
    osm_query_crs=target_osm_crs,
    used_osm_indices=None,
    line_buffer=10,
    polygon_detection_buffer=1,
)

In [ ]:
plot_area_with_sliders(
    original_border,
    new_border,
    difference_area,
    (255, 0, 0),
    base_features,
    (0, 0, 255),
    0.6,
    source_crs="EPSG:3857",
    target_crs="EPSG:4326",
)

In [ ]:
# aoi_gdf = gpd.GeoDataFrame([1], geometry=[original_geom], crs="EPSG:4326")
# aoi_gdf = aoi_gdf.explode()
# aoi_polygon_for_osmnx = aoi_gdf.geometry.iloc[0].boundary.buffer(0.05)
# gdf_polygons = ox.features_from_polygon(aoi_polygon_for_osmnx, tags)

In [ ]:
original_geom = loads(data["geometry"][4])

aoi_gdf = gpd.GeoDataFrame([1], geometry=[original_geom], crs="EPSG:4326")
aoi_gdf = aoi_gdf.explode()
aoi_polygon_for_osmnx = aoi_gdf.geometry.iloc[0].boundary.buffer(0.00001)
gdf_polygons = ox.features_from_polygon(aoi_polygon_for_osmnx, input_tags)
osm_features_proj = unary_union(gdf_polygons["geometry"].to_crs("EPSG:3857"))

original_df = gpd.GeoDataFrame([1], geometry=[original_geom], crs=initial_crs)
original_proj = original_df.to_crs("EPSG:3857")
# aligned_result_df, diff_df, process_result

In [ ]:
original_border = aoi_gdf["geometry"].to_crs("EPSG:3857")[0]

In [ ]:
# osm_features_proj

In [ ]:
# gdf_polygons.crs

In [ ]:
# type(combined_osm)
# combined_osm.explode()

In [ ]:
# type(osm_features_proj)

In [ ]:
# type(diff_df["geometry"][0])

In [ ]:
# x_coords, y_coords = aligned_result_df['geometry'].to_crs("EPSG:3857")[0].boundary.coords.xy

In [ ]:
# aligned_result_df['geometry'].crs

In [ ]:
# polygon_osgb

In [ ]:
import numpy as np  # Using numpy to easily create opacity steps
import plotly.graph_objects as go

# 1. Define Polygon Vertices
x_coords = [1, 4, 2, 1]  # X coordinates of vertices
y_coords = [1, 1, 4, 1]  # Y coordinates of vertices

# 2. Define Base Color (RGB part) and Initial Opacity
base_rgb_color = (100, 100, 220)  # e.g., Light blue
initial_opacity = 0.6
num_steps = 11  # Creates steps for 0.0, 0.1, ..., 1.0

# --- Create the initial RGBA color string ---
initial_fill_color = f"rgba({base_rgb_color[0]}, {base_rgb_color[1]}, {base_rgb_color[2]}, {initial_opacity})"
base_line_color = f"rgb({base_rgb_color[0]}, {base_rgb_color[1]}, {base_rgb_color[2]})"  # Slightly darker or same color for line


# 3. Create the Scatter trace with initial opacity
# Note: We only define one trace here (index 0)
polygon_trace = go.Scatter(
    x=list(x_coords),
    y=list(y_coords),
    fill="toself",
    mode="lines",
    line=dict(color=base_line_color),
    fillcolor=initial_fill_color,  # Set the initial fill color
    name="Polygon",  # Name in legend (optional)
)

# 4. Create Slider Steps
steps = []
opacity_values = np.linspace(
    0, 1, num_steps
)  # Generate opacity values (0.0, 0.1, ..., 1.0)

for i, op_val in enumerate(opacity_values):
    opacity_step = round(op_val, 2)  # Round for cleaner display
    # Define the RGBA color string for this step's opacity
    step_fill_color = f"rgba({base_rgb_color[0]}, {base_rgb_color[1]}, {base_rgb_color[2]}, {opacity_step})"

    step = dict(
        method="restyle",  # Method to update properties
        args=[{"fillcolor": [step_fill_color]}, [0]],
        # Arg 1: Dictionary of properties to change.
        #        Value must be a list (one item per trace targeted).
        # Arg 2: List of trace indices to apply the change to (optional, defaults might work but explicit is safer). Here [0] targets our polygon_trace.
        label=str(opacity_step),  # Label displayed on the slider tick
    )
    steps.append(step)

# 5. Find the index corresponding to the initial opacity to set the slider's starting position
initial_step_index = np.abs(opacity_values - initial_opacity).argmin()

# 6. Create the Slider dictionary
sliders = [
    dict(
        active=initial_step_index,  # Index of the initially active step (matches initial_opacity)
        currentvalue={"prefix": "Opacity: ", "visible": True},  # Display current value
        pad={"t": 50},  # Padding above the slider
        steps=steps,  # Assign the list of steps we created
    )
]

# 7. Define the Layout, including the slider
layout = go.Layout(
    title="Polygon with Opacity Controlled by Slider",
    xaxis=dict(range=[0, 5], title="X-Axis"),
    yaxis=dict(range=[0, 5], title="Y-Axis"),
    # Optional: Enforce equal aspect ratio
    # yaxis_scaleanchor="x",
    # yaxis_scaleratio=1,
    sliders=sliders,  # Add the slider configuration to the layout
)

# 8. Create and Show the Figure
fig = go.Figure(data=[polygon_trace], layout=layout)
fig.show()

In [ ]:
# combined_osm[com]
# combined_osm[0]
from shapely import MultiPolygon

combined_osm = unary_union(osm_features_proj).intersection(diff_df["geometry"])
combined_osm = combined_osm.explode()
combined_osm = combined_osm[
    combined_osm.geometry.geom_type.isin(["Polygon", "MultiPolygon"])
]
combined_osm = MultiPolygon(list(combined_osm))

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pyproj
from shapely.geometry import MultiPolygon, Polygon  # Import MultiPolygon

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# 1. DEFINE YOUR MULTIPOLYGON AND ITS ORIGINAL CRS HERE
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

# EXAMPLE: Replace this with your actual Shapely MultiPolygon object
# Creating a dummy MultiPolygon with two squares in OSGB36 (EPSG:27700)
source_crs = "EPSG:27700"
poly1 = Polygon(
    [
        (614500, 156500),
        (615500, 156500),
        (615500, 157500),
        (614500, 157500),
        (614500, 156500),
    ]
)
poly2 = Polygon(
    [
        (616000, 156000),
        (617000, 156000),
        (617000, 157000),
        (616000, 157000),
        (616000, 156000),
    ]
)
multi_polygon_osgb = MultiPolygon([poly1, poly2])

multi_polygon_osgb = osm_features_proj
multi_polygon_osgb = combined_osm
source_crs = "EPSG:3857"
# YOUR MULTIPOLYGON HERE:
# your_shapely_multipolygon = ... (load or create your MultiPolygon object)
# your_source_crs = "EPSG:XXXX"
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

# 2. Define Target CRS and Transformer
target_crs = "EPSG:4326"
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# --- --- --- MODIFIED COORDINATE EXTRACTION --- --- ---
# 3. Extract coordinates from ALL polygons in the MultiPolygon, separating with None
all_lons_wgs84 = []
all_lats_wgs84 = []

# Iterate through each Polygon inside the MultiPolygon
for poly in multi_polygon_osgb.geoms:  # Replace multi_polygon_osgb with your variable
    # Extract original coordinates for this specific polygon
    x_original, y_original = poly.boundary.coords.xy

    # Transform coordinates to WGS84 (Lon/Lat)
    lon_wgs84, lat_wgs84 = transformer.transform(x_original, y_original)

    # Append coordinates to the master lists
    all_lons_wgs84.extend(list(lon_wgs84))
    all_lats_wgs84.extend(list(lat_wgs84))

    # *** Add None separator after each polygon's coordinates ***
    all_lons_wgs84.append(None)
    all_lats_wgs84.append(None)
# --- --- --- END MODIFIED EXTRACTION --- --- ---

# 4. Calculate Centroid for Map Centering (MultiPolygon.centroid works)
centroid_original = (
    multi_polygon_osgb.centroid
)  # Replace multi_polygon_osgb with your variable
center_lon, center_lat = transformer.transform(centroid_original.x, centroid_original.y)

# --- SLIDER AND OPACITY SETUP (No changes needed here) ---
# 5. Define Base Color, Initial Opacity, and Slider Steps
base_rgb_color = (0, 100, 200)
initial_opacity = 0.6
num_steps = 11
initial_fill_color = f"rgba({base_rgb_color[0]}, {base_rgb_color[1]}, {base_rgb_color[2]}, {initial_opacity})"
line_color_rgb = (
    f"rgb({base_rgb_color[0]}, {base_rgb_color[1]-50}, {base_rgb_color[2]-100})"
)

# 6. Create Slider Steps dynamically
steps = []
opacity_values = np.linspace(0, 1, num_steps)
for i, op_val in enumerate(opacity_values):
    opacity_step = round(op_val, 2)
    step_fill_color = f"rgba({base_rgb_color[0]}, {base_rgb_color[1]}, {base_rgb_color[2]}, {opacity_step})"
    step = dict(
        method="restyle",
        # Args still targets trace 0, which now represents the whole MultiPolygon
        args=[{"fillcolor": [step_fill_color]}, [0]],
        label=str(opacity_step),
    )
    steps.append(step)

# 7. Find the index for the initial opacity step
initial_step_index = np.abs(opacity_values - initial_opacity).argmin()

# 8. Create the Slider configuration
sliders = [
    dict(
        active=initial_step_index,
        currentvalue={"prefix": "Opacity: ", "visible": True},
        pad={"t": 50},
        steps=steps,
    )
]
# --- END SLIDER SETUP ---

# 9. Create the Plotly Figure and ADD SINGLE TRACE for the MultiPolygon
fig = go.Figure()

fig.add_trace(
    go.Scattermap(
        lon=all_lons_wgs84,  # Use the combined list with None separators
        lat=all_lats_wgs84,  # Use the combined list with None separators
        mode="lines",
        fill="toself",
        fillcolor=initial_fill_color,  # Initial color for the whole trace
        line=dict(color=line_color_rgb, width=2),
        name="MultiPolygon Area",  # Single name for the combined trace
    )
)

# 10. Update Layout: Map Configuration AND Slider (No changes needed here)
initial_zoom = 15  # Adjust as needed

fig.update_layout(
    title_text=f'MultiPolygon with Opacity Slider (EPSG:{source_crs.split(":")[-1]} projected)',
    geo_scope="europe",
    map=dict(
        style="open-street-map",
        center=dict(lon=center_lon, lat=center_lat),
        zoom=initial_zoom,
    ),
    showlegend=False,
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    sliders=sliders,  # Add the slider configuration
)

# 11. Show the Figure
fig.show()

In [ ]:
def polygon_prep(
    polygon,
    transformer,  # See if this needs to be moved inside
):
    # Need to convert from crs to co-ords for this plotly method
    lons = []
    lats = []

    # Shortcut to avoid separate functions
    print(f"polygon: {polygon}")
    try:
        if polygon.geom_type == "Polygon":
            polygon = MultiPolygon([polygon])
    except:
        pass
    for poly in polygon.geoms:
        x_coords, y_coords = poly.boundary.coords.xy
        lon, lat = transformer.transform(x_coords, y_coords)
        lons.extend(list(lon))
        lats.extend(list(lat))
        # Need to add separator for multiple polygons
        lons.append(None)
        lats.append(None)

    return lons, lats

In [ ]:
def plot_area_with_sliders(
    original_border,
    new_border,
    difference_area,
    diff_rgb,
    base_features,
    base_rgb,
    alpha,
    source_crs="EPSG:3857",
    target_crs="EPSG:4326",
):
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    diff_lons, diff_lats = polygon_prep(difference_area, transformer)
    feature_lons, feature_lats = polygon_prep(base_features, transformer)

    original_lons, original_lats = polygon_prep(original_border, transformer)
    new_lons, new_lats = polygon_prep(new_border, transformer)

    boundary_center = original_border.centroid
    center_lon, center_lat = transformer.transform(boundary_center.x, boundary_center.y)

    diff_fill_colour = f"rgba({diff_rgb[0]}, {diff_rgb[1]}, {diff_rgb[2]}, {alpha})"
    diff_line_colour = f"rgba({diff_rgb[0]}, {diff_rgb[1]}, {diff_rgb[2]}, {0.6})"
    feature_fill_colour = f"rgba({base_rgb[0]}, {base_rgb[1]}, {base_rgb[2]}, {alpha})"
    feature_line_colour = f"rgba({base_rgb[0]}, {base_rgb[1]}, {base_rgb[2]}, {0.6})"

    diff_steps = []
    feature_steps = []
    alphas = np.linspace(0, 1, 11)
    for i, alpha_step in enumerate(alphas):
        alpha_step = round(alpha_step, 2)
        diff_step_color = (
            f"rgba({diff_rgb[0]}, {diff_rgb[1]}, {diff_rgb[2]}, {alpha_step})"
        )
        feature_step_color = (
            f"rgba({base_rgb[0]}, {base_rgb[1]}, {base_rgb[2]}, {alpha_step})"
        )

        diff_step = dict(
            method="restyle",
            args=[{"fillcolor": [diff_step_color]}, [0]],
            label=str(alpha_step),
        )
        diff_steps.append(diff_step)

        feature_step = dict(
            method="restyle",
            args=[{"fillcolor": [feature_step_color]}, [1]],
            label=str(alpha_step),
        )
        feature_steps.append(feature_step)

    diff_sliders = [
        dict(
            active=6,
            currentvalue={"prefix": "Added Area alpha: ", "visible": True},
            pad={"t": 20},
            steps=diff_steps,
        )
    ]

    feature_sliders = [
        dict(
            active=6,
            currentvalue={"prefix": "Base Features alpha: ", "visible": True},
            pad={"t": 120},
            steps=feature_steps,
        )
    ]

    fig = go.Figure()
    fig.add_trace(
        go.Scattermap(
            lon=diff_lons,
            lat=diff_lats,
            mode="lines",
            fill="toself",
            fillcolor=diff_fill_colour,
            line=dict(color=diff_line_colour, width=1),
            name="Concerning Areas",
            showlegend=True,
        )
    )

    fig.add_trace(
        go.Scattermap(
            lon=feature_lons,
            lat=feature_lats,
            mode="lines",
            fill="toself",
            fillcolor=feature_fill_colour,
            line=dict(color=feature_line_colour, width=1),
            name="Base Features",
            showlegend=True,
        )
    )

    fig.add_trace(
        go.Scattermap(
            lon=original_lons,
            lat=original_lats,
            mode="lines",
            fill="none",
            line=dict(color="black", width=1),
            name="Original Boundary",
            showlegend=True,
        )
    )

    fig.add_trace(
        go.Scattermap(
            lon=new_lons,
            lat=new_lats,
            mode="lines",
            fill="none",
            line=dict(color="red", width=1),
            name="New Boundary",
            showlegend=True,
        )
    )

    initial_zoom = 15

    satellite_layer = [
        dict(
            below="traces",
            sourcetype="raster",
            source=[
                "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
            ],
        )
    ]

    map_switch = [
        dict(
            args=[{"map.style": "open-street-map", "map.layers": None}],
            label="OSM",
            method="relayout",
        ),
        dict(
            args=[{"map.style": None, "map.layers": satellite_layer}],
            label="Satellite",
            method="relayout",
        ),
    ]

    fig.update_layout(
        title_text=f"MultiPolygon with Opacity Slider (EPSG:projected)",
        geo_scope="europe",
        map=dict(
            style="open-street-map",
            center=dict(lon=center_lon, lat=center_lat),
            zoom=initial_zoom,
        ),
        showlegend=True,
        margin={"r": 100, "t": 50, "l": 100, "b": 50},
        sliders=diff_sliders + feature_sliders,
        height=800,
        width=1200,
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=map_switch,
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.7,
                xanchor="right",
                y=1.15,
                yanchor="top",
            ),
        ],
    )

    fig.show()

In [ ]:
# map=dict(
#         style="open-street-map",
#         # style="https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
#         center=dict(lon=center_lon, lat=center_lat),
#         zoom=initial_zoom
#         ),
# map=dict(
#         center=dict(lon=center_lon, lat=center_lat),
#         zoom=initial_zoom,
#         # style = 'white-bg',
#         layers=[
#                 dict(
#                         below='traces',
#                         sourcetype="raster",
#                         source=["https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"],
#                     )
#                 ]
#         ),

In [ ]:
osm_features_proj

In [ ]:
plot_area_with_sliders(
    original_border,
    aligned_result_df["geometry"][0],
    combined_osm,
    (255, 0, 0),
    osm_features_proj,
    (0, 0, 255),
    0.6,
)